In [3]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os

In [7]:
files_80L = [
    'IE00031.1.csv',
    'IE00031.2.csv',
    'IE00031.3.csv'
]

fdir = 'C:\\Users\\Nathan\\Desktop\\IE00031 Raw'

files_80L = [os.path.join(fdir, f) for f in files_80L]
file_3L = os.path.join(fdir, "3L_data.csv")

In [87]:
def find(ws, v):
    # the topleft corner isn't searched by Excel. Apparently this is by design. 
    # https://docs.microsoft.com/en-us/office/vba/api/excel.range.find
    c1 = ws.Cells(1,1)
    if c1.Value == v:
        return c1
    return ws.Cells.Find(What=v, SearchOrder=xlc.xlByRows)

def data_corners(ws, v):
    cr = ws.Cells.Range
    c = find(ws, v)
    topleft = c.Offset(2, 1)
    botleft = topleft.End(xlc.xlDown)
    topright = topleft.Offset(1, 2)
    botright = botleft.Offset(1, 2)
    return topleft, botleft, topright, botright

def data_range(ws, v):
    c1, c2, c3, c4 = data_corners(ws, v)
    cr = ws.Cells.Range
    return cr(c1, c2), cr(c3, c4)

def data(ws, v):
    a,b = data_range(ws, v)
    return ws.Cells.Range(a,b).Value

def plot(ws, v, title):
    c = CreateChart(ws)
    xr, yr = data_range(ws, v)
    CreateDataSeries(c, xr, yr)
    FormatChart(c, None, title, "Time", v, None, False)
    c.Parent.Height = 291
    c.Parent.Width = 665
    c.Axes(xlc.xlCategory).TickLabels.Orientation = 10
    return c

def plot(ws, vars, title):
    c = CreateChart(ws)
    for v in vars:
        xr, yr = data_range(ws, v)
        CreateDataSeries(c, xr, yr)
    FormatChart(c, None, title, "Time", "Temp(C)", None, False)    
    c.Axes(xlc.xlCategory).TickLabels.Orientation = 10

def batch_name(ws):
    return find(ws, "Batch Name").Offset(2, 1).Value

def paste(ws, topleft, data):
    cr = ws.Cells.Range
    br = topleft.Offset(len(data), len(data[0]))
    cr(topleft, br).Value = data
    

In [88]:
def last_less(it, v):
    idx = first_greater(it, v)
    return max(idx - 2, 0)

def first_greater(it, v):
    idx = next((i for i, (x,y) in enumerate(it) if x > v), len(it))
    return idx

In [89]:
def data_by_ts(data, first, last):
    i1 = first_greater(data, first)
    i2 = last_less(data, last)
    return data[i1:i2]

In [91]:
xl = Excel()
with screen_lock(xl):
    wb3L = xl.Workbooks.Open(file_3L)
    ws3L = wb3L.Worksheets(1)
    heater_data = data(ws3L, "FilterOvenPV(C)")
    water_data = data(ws3L, "TempPV(C)")

    for fp in files_80L:
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells
        tl, bl, tr, br = data_corners(ws, "TempPV(C)")
        first_timestamp = tl.Value
        last_timestamp = bl.Value

        hdata = data_by_ts(heater_data, first_timestamp, last_timestamp)
        wdata = data_by_ts(water_data, first_timestamp, last_timestamp)

        # Insert data, using the "Batch Name" cell as reference on where to insert
        bncell = cells.Find("Batch Name", SearchOrder=xlc.xlByRows)
        bnc = bncell.Column
        bnr = bncell.Row
        bncol = bncell.EntireColumn
        for i in range(6):
            bncol.Insert()

        paste(ws, cells(bnr + 1, bnc), hdata)
        cells.Range(cells(bnr, bnc), cells(bnr, bnc+1)).Value = [('Heater Temp(C)', 'Heater Temp(c)')]

        bnc += 3
        paste(ws, cells(bnr + 1, bnc), wdata)
        cells.Range(cells(bnr, bnc), cells(bnr, bnc+1)).Value = [('Water Temp(C)', 'Water Temp(c)')]
    
        vars = [
            "TempPV(C)",
            "Heater Temp(C)",
            "Water Temp(C)"
        ]
        title = os.path.splitext(os.path.basename(fp))[0]
        plot(ws, vars, title)
    

In [4]:
xl = Excel()
with screen_lock(xl):
    for f in files:
        wb = xl.Workbooks.Open(f)
        ws = wb.Worksheets(1)
        bn = batch_name(ws)
        wb.Name = bn
        
        ttitle = "%s - %s" 
        
        plot(ws, "DOPV(%)", )
        plot(ws, "pHPV", ttitle % ("pHPV", bn))
        wb.SaveAs("D:\\%s.xlsx"%bn)

In [8]:
with screen_lock(xl):
    for wb in xl.Workbooks:
        ws = wb.Worksheets(1)
        for co in ws.ChartObjects():
            co.Chart.Location(xlc.xlLocationAsNewSheet)
        for c in wb.Charts:
            c.PrintOut()